In [4]:
#notebook to download the csv of edges and nodes of a given network
import os
import requests 
import time
import string
import networkx as nx
import itertools
import networkx as nx
import pandas as pd
import json
import re
import math
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.corpus import stopwords

from nltk.corpus import wordnet as wn #importing it
from nltk.stem.wordnet import WordNetLemmatizer #importing wordnet lemmatizer
from nltk import pos_tag #part-of-speech-tagger
from collections import defaultdict #defaultdict returns default value for non-existant keys you try to  access based on the function you passed in the constructor
from google.colab import files

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [13]:
def extract_text(df):       #extract the text from the tweets and RT
                            #works ONLY on .csv file
  list_strings = []
  for index in range(len(df)):
    if index % 1000 == 0:
      print(str(index)+' / '+str(len(df)))
    text = df.loc[index]['text']                          #if it is nor trucated nor a RT  i take "text"
    string = -1
    if (df.loc[index,"truncated"] == True):                 #if it is trucated I take "extended_tweet"
        string = df.loc[index,"extended_tweet"]
    if type(df.loc[index,"retweeted_status"]) != float:     #if it is a RT I take retweeted_status
        string = df.loc[index,"retweeted_status"]
    if type(string) == str :
        if(re.search('full_text\':(.+?)https',string) != None):     #if I find "full_text"
          s = re.search('full_text\':(.+?)https',string).group(1)
        if(re.search('text\':(.+?)https',string)!= None):
          s = re.search('text\':(.+?)https',string).group(1)
        else: 
          continue
        list_strings.append(s)
        #print(s)         
    else:
      list_strings.append(text)
      #print(text)
      

  return list_strings

In [14]:
# Cleaning, lemmatising and pos tagging tweets

nltk.download('words')
WORDS = set(nltk.corpus.words.words()) #the last two lines serve to download the corpus of standard English language words
nltk.download('stopwords') #downloading stopwords
STOP_WORDS = set(nltk.corpus.stopwords.words("english")) #taking the stop words from English language
nltk.download('wordnet') #downloading wordnet
nltk.download('averaged_perceptron_tagger') #downloading tagger
tag_map = defaultdict(lambda : wn.NOUN) #here we define that wn.NOUN is the default value for the dict
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV

def lemma_pos_cleaner(tweet):

    tweet = re.sub("@[A-Za-z0-9]+","",tweet) # remove mentions
    tweet = re.sub("#[A-Za-z0-9]+", "",tweet) # remove hashtags
    tweet = re.sub(r"(?:\@|http?\://|https?\://|www)\S+", "", tweet) # remove http links
    tweet = " ".join(tweet.split())
    tweet = str.lower(tweet) #to lowercase 
    tweet = re.sub("'"," ",tweet) # remove aphostrophe

    #basically we use pos_tag function on tokens that we get by applying wordpunct tokenization
    #to tweet (it separates all the words and symbols)
    #then we pass the token along with it's wordnet pos value that we get from the tag_map dictionary (noun, adjective, verb or adverb) to the lemma function (the WordNetLemmatizer())
    lemma_function = WordNetLemmatizer()
    tweet = " ".join(lemma_function.lemmatize(token, tag_map[tag[0]]) for token, tag in nltk.pos_tag(nltk.wordpunct_tokenize(tweet))) #lemmatize
  

    # francesco: I removed also all 2 letters words and added specific words, words that appears frequently but are discarded because they are not in the english language
    SPECIFIC_WORDS = ['virus', 'coronavirus', 'covid19', 'covid', 'trump', 'hubei', 'beijing', 'xinjiang', 'jinping', 'korea', 'xinhua', 'india', 'taiwan','johnson','singapore', 'africa', 'japanese', 'france', 'asian', 'australia', 'french', 'asia', 'leishenshan', 'british', 'qingdao', 'fauci', 'america',  'california', 'sichuan', 'malaysia', 'huawei','thailand', 'shandong', 'italy', 'philippines', 'germany', 'facebook', 'african', 'shenzhen', 'tokyo', 'russian','uygur', '5g', 'pompeo', 'vietnam', 'australian', 'cambodia', 'zhejiang', 'yunnan', 'guangdong', 'korean', 'iran', 'washington']
    tweet = " ".join(w for w in nltk.wordpunct_tokenize(tweet) if (w in WORDS or w in SPECIFIC_WORDS) and len(w)>2 and w not in STOP_WORDS ) #remove stop words
   
    return tweet

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [7]:
def frequency_dictionary(df):
  unique_words = {}

  for row in df:
    for word in row.split():
      #if the word is encountered for the first time add to dict as key and set its value to 0
      unique_words.setdefault(word,0)
      #increase the value (i.e the count) of the word by 1 every time it is encountered
      unique_words[word] += 1

  return unique_words

In [133]:
period = ''  # _JanFeb2020, _MarchApril2021, _SeptOct2020

In [134]:
China = pd.read_csv('/content/China'+period+'.csv')
USA = pd.read_csv('/content/USA'+period+'.csv')
China_USA = pd.read_csv('/content/China&USA'+period+'.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (34,35,38) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (38) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [135]:
# number of tweets:
print('China: ', len(China))
print('USA: ', len(USA))
print('China&USA: ', len(China_USA))

China:  9350
USA:  19865
China&USA:  29215


In [130]:
text_China = extract_text(China)
text_USA = extract_text(USA)
text_China_USA = extract_text(China_USA)

0 / 4876
1000 / 4876
2000 / 4876
3000 / 4876
4000 / 4876
0 / 7519
1000 / 7519
2000 / 7519
3000 / 7519
4000 / 7519
5000 / 7519
6000 / 7519
7000 / 7519
0 / 12395
1000 / 12395
2000 / 12395
3000 / 12395
4000 / 12395
5000 / 12395
6000 / 12395
7000 / 12395
8000 / 12395
9000 / 12395
10000 / 12395
11000 / 12395
12000 / 12395


In [16]:
cleaned_text_China = [lemma_pos_cleaner(txt) for txt in text_China]
cleaned_text_USA = [lemma_pos_cleaner(txt) for txt in text_USA]
cleaned_text_China_USA = [lemma_pos_cleaner(txt) for txt in text_China_USA]

print('China:')
print(cleaned_text_China[0:10])
print()
print('USA:')
print(cleaned_text_USA[0:10])
print()
print('China&USA:')
print(cleaned_text_China_USA[0:10])

China:
['talk university challenge china face deal misconception people medium watch', 'spring always bring along lively saturate sea colorful bloom flower look collection flower blossom across china enjoy upcoming new season', 'mean carnival brazil year one samba school take inspiration china', 'china world make great stride track infectious disease take unexpected turn make hard track', 'fake news story case covid lead protest evacuation people', 'transmission outside china graphic', 'world health organization announce lead team international expert currently china', 'outside china confirm case novel covid half cruise ship dock japan rest scatter among country mostly asia', 'five cargo aircraft airway head city beijing shanghai carry ton medical supply gratis include million mask hand send epidemic prevention supply purchase china government', 'case miss jinping chair lead body meeting covid control']

USA:
['shameful attack bus carry evacuee china coronavirus outbreak', 'china hubei

In [73]:
freq_dict_China = frequency_dictionary(cleaned_text_China)
freq_dict_China = dict(sorted(freq_dict_China.items(), key=lambda item: item[1], reverse = True))   #order from more frequent to less frequent word

freq_dict_USA = frequency_dictionary(cleaned_text_USA)
freq_dict_USA = dict(sorted(freq_dict_USA.items(), key=lambda item: item[1], reverse = True))   #order from more frequent to less frequent word

freq_dict_China_USA = frequency_dictionary(cleaned_text_China_USA)
freq_dict_China_USA = dict(sorted(freq_dict_China_USA.items(), key=lambda item: item[1], reverse = True))   #order from more frequent to less frequent word

# number of words in the cleaned tweets:
print('China: ', len(list(freq_dict_China)))
print('USA: ', len(list(freq_dict_USA)))
print('China&USA: ', len(list(freq_dict_China_USA)))

China:  5699
USA:  6476
China&USA:  8185


In [74]:
# Most frequent words
print('China')
print([key for key in freq_dict_China.keys() if freq_dict_China[key]>200])
print()
print('USA')
print([key for key in freq_dict_USA.keys() if freq_dict_USA[key]>400])
print()
print('China&USA')
print([key for key in freq_dict_China_USA.keys() if freq_dict_China_USA[key]>600])

China
['china', 'covid', 'novel', 'case', 'coronavirus', 'say', 'new', 'hospital', 'vaccine', 'outbreak', 'people', 'report', 'patient', 'fight', 'country', 'health', 'day', 'city', 'province', 'year', 'confirm', 'first', 'world', 'epidemic', 'death', 'live', 'national', 'medical', 'president', 'amid', 'million', 'hubei', 'one', 'take', 'support', 'virus', 'control', 'battle', 'help', 'test', 'see', 'pandemic', 'official', 'infection', 'show', 'work', 'number', 'effort', 'global', 'two', 'late', 'expert', 'receive', 'make', 'total', 'central', 'time', 'watch', 'accord', 'use', 'measure', 'state', 'across', 'high', 'discharge', 'bring', 'development', 'infect']

USA
['coronavirus', 'covid', 'china', 'say', 'vaccine', 'case', 'new', 'outbreak', 'virus', 'test', 'trump', 'death', 'people', 'report', 'president', 'health', 'spread', 'first', 'state', 'country', 'day', 'world', 'pandemic', 'million', 'positive', 'rise', 'late', 'year', 'infection', 'city', 'global', 'official', 'week', 'two

# less frequent words

In [75]:
# less frequent words:
print('Less frequent China: ', len([key for key in freq_dict_China.keys() if freq_dict_China[key]<10]))
print('More frequent China: ', len([key for key in freq_dict_China.keys() if freq_dict_China[key]>=10]))
less_frequent_words_China = [key for key in freq_dict_China.keys() if freq_dict_China[key]<10]

print('Less frequent USA: ', len([key for key in freq_dict_USA.keys() if freq_dict_USA[key]<10]))
print('More frequent USA:', len([key for key in freq_dict_USA.keys() if freq_dict_USA[key]>=10]))
less_frequent_words_USA = [key for key in freq_dict_USA.keys() if freq_dict_USA[key]<10]

print('Less frequent China&USA: ', len([key for key in freq_dict_China_USA.keys() if freq_dict_China_USA[key]<10]))
print('More frequent China&USA: ', len([key for key in freq_dict_China_USA.keys() if freq_dict_China_USA[key]>=10]))
less_frequent_words_China_USA = [key for key in freq_dict_China_USA.keys() if freq_dict_China_USA[key]<10]

Less frequent China:  4155
More frequent China:  1544
Less frequent USA:  4287
More frequent USA: 2189
Less frequent China&USA:  5344
More frequent China&USA:  2841


In [43]:
# discard less frequent words
cleaned_mostfreq_text_China = cleaned_text_China.copy()
for txt in range(len(cleaned_mostfreq_text_China)):
  if txt % 1000 == 0:
    print(txt, '/',len(cleaned_mostfreq_text_China))
  for word in less_frequent_words_China:
    if word in cleaned_mostfreq_text_China[txt].split():
      cleaned_mostfreq_text_China[txt] = cleaned_mostfreq_text_China[txt].replace(word, '')
      cleaned_mostfreq_text_China[txt] = " ".join(cleaned_mostfreq_text_China[txt].split())

cleaned_mostfreq_text_USA = cleaned_text_USA.copy()
for txt in range(len(cleaned_mostfreq_text_USA)):
  if txt % 1000 == 0:
    print(txt, '/',len(cleaned_mostfreq_text_USA))
  for word in less_frequent_words_USA:
    if word in cleaned_mostfreq_text_USA[txt].split():
      cleaned_mostfreq_text_USA[txt] = cleaned_mostfreq_text_USA[txt].replace(word, '')
      cleaned_mostfreq_text_USA[txt] = " ".join(cleaned_mostfreq_text_USA[txt].split())

cleaned_mostfreq_text_China_USA = cleaned_text_China_USA.copy()
for txt in range(len(cleaned_mostfreq_text_China_USA)):
  if txt % 1000 == 0:
    print(txt, '/',len(cleaned_mostfreq_text_China_USA))
  for word in less_frequent_words_China_USA:
    if word in cleaned_mostfreq_text_China_USA[txt].split():
      cleaned_mostfreq_text_China_USA[txt] = cleaned_mostfreq_text_China_USA[txt].replace(word, '')
      cleaned_mostfreq_text_China_USA[txt] = " ".join(cleaned_mostfreq_text_China_USA[txt].split())

0 / 9268
1000 / 9268
2000 / 9268
3000 / 9268
4000 / 9268
5000 / 9268
6000 / 9268
7000 / 9268
8000 / 9268
9000 / 9268
0 / 19860
1000 / 19860
2000 / 19860
3000 / 19860
4000 / 19860
5000 / 19860
6000 / 19860
7000 / 19860
8000 / 19860
9000 / 19860
10000 / 19860
11000 / 19860
12000 / 19860
13000 / 19860
14000 / 19860
15000 / 19860
16000 / 19860
17000 / 19860
18000 / 19860
19000 / 19860
0 / 29128
1000 / 29128
2000 / 29128
3000 / 29128
4000 / 29128
5000 / 29128
6000 / 29128
7000 / 29128
8000 / 29128
9000 / 29128
10000 / 29128
11000 / 29128
12000 / 29128
13000 / 29128
14000 / 29128
15000 / 29128
16000 / 29128
17000 / 29128
18000 / 29128
19000 / 29128
20000 / 29128
21000 / 29128
22000 / 29128
23000 / 29128
24000 / 29128
25000 / 29128
26000 / 29128
27000 / 29128
28000 / 29128
29000 / 29128


In [77]:
freq_dict_China = frequency_dictionary(cleaned_mostfreq_text_China)
freq_dict_China = dict(sorted(freq_dict_China.items(), key=lambda item: item[1], reverse = True))   #order from more frequent to less frequent word

freq_dict_USA = frequency_dictionary(cleaned_mostfreq_text_USA)
freq_dict_USA = dict(sorted(freq_dict_USA.items(), key=lambda item: item[1], reverse = True))   #order from more frequent to less frequent word

freq_dict_China_USA = frequency_dictionary(cleaned_mostfreq_text_China_USA)
freq_dict_China_USA = dict(sorted(freq_dict_China_USA.items(), key=lambda item: item[1], reverse = True))   #order from more frequent to less frequent word

# number of words in the cleaned tweets:
print('China: ', len(list(freq_dict_China)))
print('USA: ', len(list(freq_dict_USA)))
print('China&USA: ', len(list(freq_dict_China_USA)))

China:  1573
USA:  2208
China&USA:  2862


# Build Network

In [78]:
df_China = pd.DataFrame.from_dict(freq_dict_China, orient='index').reset_index()
df_China.rename(columns = {'index':'Word', 0:'Count'}, inplace=True)
df_China.sort_values(by=['Count'], ascending=False, inplace=True)
df_China.reset_index(inplace=True)
df_China.drop(columns="index",inplace=True)

df_USA = pd.DataFrame.from_dict(freq_dict_USA, orient='index').reset_index()
df_USA.rename(columns = {'index':'Word', 0:'Count'}, inplace=True)
df_USA.sort_values(by=['Count'], ascending=False, inplace=True)
df_USA.reset_index(inplace=True)
df_USA.drop(columns="index",inplace=True)

df_China_USA = pd.DataFrame.from_dict(freq_dict_China_USA, orient='index').reset_index()
df_China_USA.rename(columns = {'index':'Word', 0:'Count'}, inplace=True)
df_China_USA.sort_values(by=['Count'], ascending=False, inplace=True)
df_China_USA.reset_index(inplace=True)
df_China_USA.drop(columns="index",inplace=True)

print('China')
print(df_China.iloc[0:30])
print()
print('USA')
print(df_USA.iloc[0:30])
print()
print('China&USA')
print(df_China_USA.iloc[0:30])
print()

China
           Word  Count
0         china   5087
1         covid   1749
2         novel   1402
3          case   1359
4   coronavirus   1264
5           say   1158
6           new   1128
7      hospital   1014
8       vaccine    960
9      outbreak    877
10       people    757
11       report    705
12      patient    685
13        fight    683
14      country    640
15       health    638
16          day    594
17         city    572
18     province    571
19         year    552
20      confirm    542
21        first    510
22        world    492
23     epidemic    460
24        death    446
25         live    443
26     national    442
27      medical    440
28    president    434
29         amid    396

USA
           Word  Count
0   coronavirus   7552
1         covid   7261
2         china   5151
3           say   3474
4       vaccine   3383
5          case   2560
6           new   2489
7      outbreak   1619
8         virus   1536
9          test   1424
10        trump   1396


In [79]:
keys_China = freq_dict_China.keys()  
keys_USA = freq_dict_USA.keys()  
keys_China_USA = freq_dict_China_USA.keys()  

In [80]:
def create_network(cleaned_text):
  network = {}
  #connect the word that appear in the same tweets
  for row in cleaned_text:
    combined_list = [word for word in str.split(row)]
    #for pair in itertools.product(combined_list, combined_list):
    #print(combined_list)
    for pair in itertools.product(combined_list, combined_list):
          #exclude self-loops and count each pair only once because our graph is undirected and we do not take self-loops into account
          if pair[0]!=pair[1] and not(pair[::-1] in network):
              network.setdefault(pair,0)
              network[pair] += 1 
  network_df = pd.DataFrame.from_dict(network, orient="index")
  network_df.columns = ["weight"]
  network_df.sort_values(by="weight",inplace=True, ascending=False)
  return network, network_df

In [81]:
network_China, network_df_China = create_network(cleaned_mostfreq_text_China)
network_USA, network_df_USA = create_network(cleaned_mostfreq_text_USA)
network_China_USA, network_df_China_USA = create_network(cleaned_mostfreq_text_China_USA)

In [82]:
print('China:')
print(network_df_China.iloc[0:30])
print()
print('USA:')
print(network_df_USA.iloc[0:30])
print()
print('China&USA:')
print(network_df_China_USA.iloc[0:30])
print()

China:
                      weight
(china, novel)          1025
(china, say)             762
(new, case)              758
(china, coronavirus)     662
(confirm, case)          653
(china, outbreak)        650
(case, report)           632
(china, case)            602
(vaccine, covid)         586
(china, new)             577
(patient, hospital)      572
(china, fight)           572
(china, covid)           547
(novel, coronavirus)     499
(china, province)        441
(report, new)            434
(case, death)            432
(china, people)          415
(epidemic, china)        401
(novel, outbreak)        394
(china, country)         393
(china, support)         385
(china, national)        356
(year, china)            352
(china, hospital)        348
(day, china)             343
(city, china)            339
(case, covid)            333
(china, vaccine)         329
(china, confirm)         324

USA:
                         weight
(china, coronavirus)       2665
(vaccine, covid)        

#Graph


In [83]:
def get_graph(network):
  up_weighted = []
  for edge in network:
      #we can filter edges by weight by uncommenting the next line and setting desired weight threshold
      up_weighted.append((edge[0],edge[1],network[edge]))
      
  #print(network)
  #print(up_weighted[0:10])
  G = nx.Graph()
  G.add_weighted_edges_from(up_weighted)
  return G

In [84]:
G_China = get_graph(network_China)
G_USA = get_graph(network_USA)
G_China_USA = get_graph(network_China_USA)

In [85]:
print('China:')
print('Nodes: ',len(G_China.nodes()))
print('Edges: ',len(G_China.edges()))
print('Is connected: ',nx.is_connected(G_China))
print()
print('USA:')
print('Nodes: ',len(G_USA.nodes()))
print('Edges: ',len(G_USA.edges()))
print('Is connected: ',nx.is_connected(G_USA))
print()
print('China&USA:')
print('Nodes: ',len(G_China_USA.nodes()))
print('Edges: ',len(G_China_USA.edges()))
print('Is connected: ',nx.is_connected(G_China_USA))

China:
Nodes:  1572
Edges:  155952
Is connected:  True

USA:
Nodes:  2208
Edges:  241398
Is connected:  True

China&USA:
Nodes:  2862
Edges:  391951
Is connected:  True


# PageRank

In [86]:
# Calculating the pagerank on graph G, teleportation probability here is 0.15 but since the graph is strongly connected we can set it to zero if we want
pr_China = nx.algorithms.pagerank(G_China,alpha = 1)
pr_China = dict(sorted(pr_China.items(), key=lambda item: item[1],reverse  = True))

pr_USA = nx.algorithms.pagerank(G_USA,alpha = 1)
pr_USA = dict(sorted(pr_USA.items(), key=lambda item: item[1],reverse  = True))

pr_China_USA = nx.algorithms.pagerank(G_China_USA,alpha = 1)
pr_China_USA = dict(sorted(pr_China_USA.items(), key=lambda item: item[1],reverse  = True))

In [92]:
def threshold(vector,threshold):

  l = [(el,vector[el]) for el in vector if vector[el] >= threshold ]

  return pd.DataFrame(l)

In [106]:
def threshold_reverse(vector,threshold):

  l = [(el,vector[el]) for el in vector if vector[el] < threshold ]

  return pd.DataFrame(l)

In [104]:
print('China: ', len(threshold(pr_China,0.0003)))
print()
print('first:')
print(threshold(pr_China,0.0003).iloc[:30])
print()
print('last:')
print(threshold(pr_China,0.0003).iloc[684:])
print()
print('USA: ', len(threshold(pr_USA,0.0003)))
print()
print('first:')
print(threshold(pr_USA,0.0003).iloc[:30])
print()
print('last:')
print(threshold(pr_USA,0.0003).iloc[639:])
print()
print('China&USA: ', len(threshold(pr_China_USA,0.0003)))
print()
print('first:')
print(threshold(pr_China_USA,0.0003).iloc[:30])
print()
print('last:')
print(threshold(pr_China_USA,0.0003).iloc[648:])

China:  714

first:
              0         1
0         china  0.048910
1         covid  0.015820
2         novel  0.014042
3          case  0.013749
4           say  0.013300
5   coronavirus  0.011654
6           new  0.011312
7      hospital  0.009882
8      outbreak  0.008687
9        people  0.008277
10      vaccine  0.008052
11       report  0.007580
12       health  0.007339
13      country  0.007219
14      patient  0.006854
15          day  0.006802
16         city  0.006615
17     province  0.006574
18        fight  0.006523
19         year  0.006340
20      confirm  0.005904
21        world  0.005461
22        first  0.005375
23     national  0.005041
24      medical  0.004920
25     epidemic  0.004892
26        death  0.004846
27          one  0.004540
28    president  0.004418
29        hubei  0.004358

last:
                0         1
684        access  0.000309
685          task  0.000309
686        single  0.000309
687        reform  0.000308
688         thank  0.000308

# TF-IDF

In [61]:
tfidf = TfidfVectorizer(ngram_range=(1,1))   # ngram range can be changed to obtain measures regarding n grams instead of single words

X_China = tfidf.fit_transform(cleaned_mostfreq_text_China).toarray()    # entry (i,j) if Tfidf measure of word_list[j] in document i
word_list_China = tfidf.get_feature_names_out()

X_USA = tfidf.fit_transform(cleaned_mostfreq_text_USA).toarray()
word_list_USA = tfidf.get_feature_names_out()

X_China_USA = tfidf.fit_transform(cleaned_mostfreq_text_China_USA).toarray()
word_list_China_USA = tfidf.get_feature_names_out()


In [62]:
tfidf_df_China = pd.DataFrame(X_China,columns = word_list_China)

tfidf_df_USA = pd.DataFrame(X_USA,columns = word_list_USA)

tfidf_df_China_USA = pd.DataFrame(X_China_USA,columns = word_list_China_USA)

In [63]:
tfidf_word_measure_China = np.mean(tfidf_df_China,axis = 0)
tfidf_word_measure_China = tfidf_word_measure_China.sort_values(ascending = False)
tfidf_word_measure_USA = np.mean(tfidf_df_USA,axis = 0)
tfidf_word_measure_USA = tfidf_word_measure_USA.sort_values(ascending = False)
tfidf_word_measure_China_USA = np.mean(tfidf_df_China_USA,axis = 0)
tfidf_word_measure_China_USA = tfidf_word_measure_China_USA.sort_values(ascending = False)

In [64]:
print('China:')
print(tfidf_word_measure_China[0:30])
print()
print('USA:')
print(tfidf_word_measure_USA[0:30])
print()
print('China&USA:')
print(tfidf_word_measure_China_USA[0:30])
print()

China:
china          0.057393
covid          0.032344
case           0.029077
novel          0.027626
coronavirus    0.027115
new            0.025339
vaccine        0.022859
hospital       0.022712
say            0.020808
outbreak       0.019470
report         0.017295
fight          0.016957
patient        0.016629
people         0.015936
confirm        0.014500
watch          0.014347
health         0.014288
country        0.014000
day            0.013577
live           0.013453
province       0.012905
first          0.012837
city           0.012793
death          0.012317
year           0.012240
world          0.011623
epidemic       0.011616
amid           0.011082
medical        0.011039
president      0.010959
dtype: float64

USA:
coronavirus    0.049972
covid          0.048809
china          0.041930
vaccine        0.031179
case           0.028555
say            0.027612
new            0.026551
report         0.017860
death          0.017719
outbreak       0.017477
virus       

# reduced graph

In [113]:
# less frequent words:
less_important_words_China = [key for key in list(threshold_reverse(pr_China,0.0003)[0])]

less_important_words_USA = [key for key in list(threshold_reverse(pr_USA,0.0003)[0])]

less_important_words_China_USA = [key for key in list(threshold_reverse(pr_China_USA,0.0003)[0])]

In [114]:
# discard less frequent words
cleaned_mostimp_text_China = cleaned_mostfreq_text_China.copy()
for txt in range(len(cleaned_mostimp_text_China)):
  if txt % 1000 == 0:
    print(txt, '/',len(cleaned_mostimp_text_China))
  for word in less_important_words_China:
    if word in cleaned_mostimp_text_China[txt].split():
      cleaned_mostimp_text_China[txt] = cleaned_mostimp_text_China[txt].replace(word, '')
      cleaned_mostimp_text_China[txt] = " ".join(cleaned_mostimp_text_China[txt].split())

cleaned_mostimp_text_USA = cleaned_mostfreq_text_USA.copy()
for txt in range(len(cleaned_mostimp_text_USA)):
  if txt % 1000 == 0:
    print(txt, '/',len(cleaned_mostimp_text_USA))
  for word in less_important_words_USA:
    if word in cleaned_mostimp_text_USA[txt].split():
      cleaned_mostimp_text_USA[txt] = cleaned_mostimp_text_USA[txt].replace(word, '')
      cleaned_mostimp_text_USA[txt] = " ".join(cleaned_mostimp_text_USA[txt].split())

cleaned_mostimp_text_China_USA = cleaned_mostfreq_text_China_USA.copy()
for txt in range(len(cleaned_mostimp_text_China_USA)):
  if txt % 1000 == 0:
    print(txt, '/',len(cleaned_mostimp_text_China_USA))
  for word in less_important_words_China_USA:
    if word in cleaned_mostimp_text_China_USA[txt].split():
      cleaned_mostimp_text_China_USA[txt] = cleaned_mostimp_text_China_USA[txt].replace(word, '')
      cleaned_mostimp_text_China_USA[txt] = " ".join(cleaned_mostimp_text_China_USA[txt].split())

0 / 9268
1000 / 9268
2000 / 9268
3000 / 9268
4000 / 9268
5000 / 9268
6000 / 9268
7000 / 9268
8000 / 9268
9000 / 9268
0 / 19860
1000 / 19860
2000 / 19860
3000 / 19860
4000 / 19860
5000 / 19860
6000 / 19860
7000 / 19860
8000 / 19860
9000 / 19860
10000 / 19860
11000 / 19860
12000 / 19860
13000 / 19860
14000 / 19860
15000 / 19860
16000 / 19860
17000 / 19860
18000 / 19860
19000 / 19860
0 / 29128
1000 / 29128
2000 / 29128
3000 / 29128
4000 / 29128
5000 / 29128
6000 / 29128
7000 / 29128
8000 / 29128
9000 / 29128
10000 / 29128
11000 / 29128
12000 / 29128
13000 / 29128
14000 / 29128
15000 / 29128
16000 / 29128
17000 / 29128
18000 / 29128
19000 / 29128
20000 / 29128
21000 / 29128
22000 / 29128
23000 / 29128
24000 / 29128
25000 / 29128
26000 / 29128
27000 / 29128
28000 / 29128
29000 / 29128


In [115]:
freq_dict_China = frequency_dictionary(cleaned_mostimp_text_China)
freq_dict_China = dict(sorted(freq_dict_China.items(), key=lambda item: item[1], reverse = True))   #order from more frequent to less frequent word

freq_dict_USA = frequency_dictionary(cleaned_mostimp_text_USA)
freq_dict_USA = dict(sorted(freq_dict_USA.items(), key=lambda item: item[1], reverse = True))   #order from more frequent to less frequent word

freq_dict_China_USA = frequency_dictionary(cleaned_mostimp_text_China_USA)
freq_dict_China_USA = dict(sorted(freq_dict_China_USA.items(), key=lambda item: item[1], reverse = True))   #order from more frequent to less frequent word

# number of words in the cleaned tweets:
print('China: ', len(list(freq_dict_China)))
print('USA: ', len(list(freq_dict_USA)))
print('China&USA: ', len(list(freq_dict_China_USA)))

China:  752
USA:  710
China&USA:  766


In [116]:
df_China = pd.DataFrame.from_dict(freq_dict_China, orient='index').reset_index()
df_China.rename(columns = {'index':'Word', 0:'Count'}, inplace=True)
df_China.sort_values(by=['Count'], ascending=False, inplace=True)
df_China.reset_index(inplace=True)
df_China.drop(columns="index",inplace=True)

df_USA = pd.DataFrame.from_dict(freq_dict_USA, orient='index').reset_index()
df_USA.rename(columns = {'index':'Word', 0:'Count'}, inplace=True)
df_USA.sort_values(by=['Count'], ascending=False, inplace=True)
df_USA.reset_index(inplace=True)
df_USA.drop(columns="index",inplace=True)

df_China_USA = pd.DataFrame.from_dict(freq_dict_China_USA, orient='index').reset_index()
df_China_USA.rename(columns = {'index':'Word', 0:'Count'}, inplace=True)
df_China_USA.sort_values(by=['Count'], ascending=False, inplace=True)
df_China_USA.reset_index(inplace=True)
df_China_USA.drop(columns="index",inplace=True)

print('China')
print(df_China.iloc[0:30])
print()
print('USA')
print(df_USA.iloc[0:30])
print()
print('China&USA')
print(df_China_USA.iloc[0:30])
print()

China
           Word  Count
0         china   5087
1         covid   1749
2         novel   1402
3          case   1359
4   coronavirus   1264
5           say   1157
6           new   1128
7      hospital   1014
8       vaccine    959
9      outbreak    877
10       people    757
11       report    705
12      patient    685
13        fight    683
14      country    639
15       health    638
16          day    594
17         city    572
18     province    571
19         year    552
20      confirm    542
21        first    510
22        world    492
23     epidemic    460
24        death    446
25         live    443
26     national    442
27      medical    440
28    president    434
29         amid    396

USA
           Word  Count
0   coronavirus   7549
1         covid   7261
2         china   5151
3           say   3473
4       vaccine   3383
5          case   2560
6           new   2489
7      outbreak   1619
8         virus   1539
9          test   1423
10        trump   1396


In [117]:
keys_China = freq_dict_China.keys()  
keys_USA = freq_dict_USA.keys()  
keys_China_USA = freq_dict_China_USA.keys()  

In [118]:
network_China, network_df_China = create_network(cleaned_mostimp_text_China)
network_USA, network_df_USA = create_network(cleaned_mostimp_text_USA)
network_China_USA, network_df_China_USA = create_network(cleaned_mostimp_text_China_USA)

In [119]:
print('China:')
print(network_df_China.iloc[0:30])
print()
print('USA:')
print(network_df_USA.iloc[0:30])
print()
print('China&USA:')
print(network_df_China_USA.iloc[0:30])
print()

China:
                      weight
(china, novel)          1025
(china, say)             762
(new, case)              758
(china, coronavirus)     662
(confirm, case)          653
(china, outbreak)        650
(case, report)           632
(china, case)            602
(vaccine, covid)         585
(china, new)             577
(china, fight)           572
(patient, hospital)      572
(china, covid)           547
(novel, coronavirus)     499
(china, province)        441
(report, new)            434
(case, death)            432
(china, people)          415
(epidemic, china)        401
(novel, outbreak)        394
(china, country)         393
(china, support)         385
(china, national)        356
(year, china)            352
(china, hospital)        348
(day, china)             343
(city, china)            339
(case, covid)            333
(china, vaccine)         329
(china, confirm)         324

USA:
                         weight
(china, coronavirus)       2665
(vaccine, covid)        

In [120]:
def get_graph(network):
  up_weighted = []
  for edge in network:
      #we can filter edges by weight by uncommenting the next line and setting desired weight threshold
      up_weighted.append((edge[0],edge[1],network[edge]))
      
  #print(network)
  #print(up_weighted[0:10])
  G = nx.Graph()
  G.add_weighted_edges_from(up_weighted)
  return G

In [121]:
G_China = get_graph(network_China)
G_USA = get_graph(network_USA)
G_China_USA = get_graph(network_China_USA)

In [122]:
print('China:')
print('Nodes: ',len(G_China.nodes()))
print('Edges: ',len(G_China.edges()))
print('Is connected: ',nx.is_connected(G_China))
print()
print('USA:')
print('Nodes: ',len(G_USA.nodes()))
print('Edges: ',len(G_USA.edges()))
print('Is connected: ',nx.is_connected(G_USA))
print()
print('China&USA:')
print('Nodes: ',len(G_China_USA.nodes()))
print('Edges: ',len(G_China_USA.edges()))
print('Is connected: ',nx.is_connected(G_China_USA))

China:
Nodes:  751
Edges:  82457
Is connected:  True

USA:
Nodes:  710
Edges:  99452
Is connected:  True

China&USA:
Nodes:  766
Edges:  125167
Is connected:  True


#Save edge list

In [124]:
filename = './edgelist_China_700.csv'
nx.write_weighted_edgelist(G_China, filename, delimiter=",")
#add header with appropriate column names (works on colab and Linux/Mac(?))
!sed -i.bak 1i"Source,Target,Weight" ./edgelist_China_700.csv
files.download("edgelist_China_700.csv")

filename = './edgelist_USA_700.csv'
nx.write_weighted_edgelist(G_USA, filename, delimiter=",")
#add header with appropriate column names (works on colab and Linux/Mac(?))
!sed -i.bak 1i"Source,Target,Weight" ./edgelist_USA_700.csv
files.download("edgelist_USA_700.csv")

filename = './edgelist_China_USA_700.csv'
nx.write_weighted_edgelist(G_China_USA, filename, delimiter=",")
#add header with appropriate column names (works on colab and Linux/Mac(?))
!sed -i.bak 1i"Source,Target,Weight" ./edgelist_China_USA_700.csv
files.download("edgelist_China_USA_700.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>





# Create Node List




In [125]:
def nodes(freq_dict, name):
  word_nodes = pd.DataFrame.from_dict(freq_dict,orient="index")
  word_nodes.reset_index(inplace=True)
  word_nodes["Label"] = word_nodes["index"]
  word_nodes.rename(columns={"index":"Id",0:"delete"},inplace=True)
  word_nodes = word_nodes.drop(columns=['delete'])
  nodelist = pd.DataFrame()
  nodelist = nodelist.append(word_nodes, ignore_index=True)

  nodelist = nodelist.to_csv("nodelist_"+name+".csv",index=False)
  files.download("nodelist_"+name+".csv")
  return nodelist, word_nodes

In [126]:
nodelist_China, word_nodes_China = nodes(freq_dict_China,"China_700")
nodelist_USA, word_nodes_USA = nodes(freq_dict_USA,"USA_700")
nodelist_China_USA, word_nodes_China_USA = nodes(freq_dict_China_USA,"China_USA_700")

print('China:')
print(word_nodes_China.head())
print()
print('USA:')
print(word_nodes_USA.head())
print()
print('China&USA:')
print(word_nodes_China_USA.head())
print()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

China:
            Id        Label
0        china        china
1        covid        covid
2        novel        novel
3         case         case
4  coronavirus  coronavirus

USA:
            Id        Label
0  coronavirus  coronavirus
1        covid        covid
2        china        china
3          say          say
4      vaccine      vaccine

China&USA:
            Id        Label
0        china        china
1        covid        covid
2  coronavirus  coronavirus
3          say          say
4      vaccine      vaccine

